In [1]:
from elasticsearch import Elasticsearch
import glob
import re
import io 
import os

file_name_path = "./jsai/"

In [2]:
# ライブラリのy意味込み
import requests

# メッセージ送信、画像送信、スタンプ送信の処理をクラス化
class LINENotifyBot:
    API_URL = 'https://notify-api.line.me/api/notify'
    def __init__(self, access_token):
        self.__headers = {'Authorization': 'Bearer ' + access_token}

    def send(
            self, message,
            image=None, sticker_package_id=None, sticker_id=None,
            ):
        payload = {
            'message': message,
            'stickerPackageId': sticker_package_id,
            'stickerId': sticker_id,
            }
        files = {}
        if image != None:
            files = {'imageFile': open(image, 'rb')}
        r = requests.post(
            LINENotifyBot.API_URL,
            headers=self.__headers,
            data=payload,
            files=files,
            )

In [3]:
bot = LINENotifyBot(access_token='luTtDSuF8bgeUBZoPDyA6s44vHlQO2ptwIWGM8dW7Yp')
def LINE(text):
    bot.send(message=text)

In [4]:
file_names = os.listdir(file_name_path)

In [5]:
def dict_create(file_name_path, file_names):
    '''
    dict_create
    * file_name_path :root path
    * file_names     :file name list
    ファイルの読み込みをわかりやすくするために辞書型のデータを作成
    フォルダ名：ファイル名の関係にしてある
    '''
    data_dict = {}
    for name in file_names:
        if os.path.isdir(file_name_path + name):
            #data_dict[name] = glob.glob(file_name_path + name + "/txt/*.txt")
            name_list = os.listdir(file_name_path + name + "/txt/")
            data_dict[name] = [i for i in name_list if i[-4:] == ".txt"]

    return data_dict

In [6]:
def setting():
    settings = {
        "settings": {
            "analysis": {
                "filter": {
                    "synonyms_filter": { # 同義語フィルターの定義
                        "type": "synonym",
                        "synonyms": [ #同義語リストの定義 (今は空の状態)
                            ]
                    }
                },
                "tokenizer": {
                    "sudachi_tokenizer": {
                        "type": "sudachi_tokenizer",
                        "discard_punctuation": True,
                        "sudachi_split": "search",
                        "resources_path": "/usr/share/elasticsearch/config/sudachi",
                        "settings_path": "/usr/share/elasticsearch/config/sudachi/sudachi_fulldict.json"
                    }
                },
                "analyzer": {
                    "sudachi_analyzer": {
                        "char_filter": [
                            "icu_normalizer", # 文字単位の正規化
                            "kuromoji_iteration_mark" # 繰り返し文字の正規化
                        ],
                        "filter": [
                            "synonyms_filter", # 同義語展開
                            # "kuromoji_baseform", # 活用語の原型化
                            # "kuromoji_part_of_speech", # 不要品詞の除去
                            # "ja_stop", #不要単語の除去
                            "kuromoji_number", # 数字の正規化
                            "kuromoji_stemmer" #長音の正規化
                        ],
                        "tokenizer": "sudachi_tokenizer",
                        "type": "custom"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "category": {
                    "type": "keyword"
                },
                "title": {
                    "type": "keyword"
                },
                "text": {
                    "analyzer": "sudachi_analyzer",
                    "type": "text"
                },
                "sudachi": {
                    "type": "text"
                }
            }
        }
    }
    return settings

In [7]:
def create_es(index_name, es_host_name, setting):
    es = Elasticsearch(es_host_name)
    if es.indices.exists(index=index_name):
        print(f"{index_name}を更新します。")
        es.indices.delete(index=index_name)
    result = es.indices.create(index=index_name, body=setting)
    if result["acknowledged"]:
        print(f'create index【{result["index"]}】')
        return es

In [8]:
data_dict = dict_create(file_name_path, file_names)
es_name = "jsai_dataset"
es_host_name = "elasticsearch-sudachi"
es = create_es(es_name, es_host_name, setting())

jsai_datasetを更新します。
create index【jsai_dataset】


In [9]:
def stop_word(text):
    # 文末の改行を削除
    text_data = re.sub("\n+$", "", text)
    text_data = text_data.split(",")
    return text_data

In [10]:
def input_datas(es, index_name, category, file_names):
    for name in file_names:
        original_data = io.open(f"./jsai/{category}/txt/{name}",mode="r", encoding="utf-8").read()
        datas = stop_word(original_data)
        for num, data in enumerate(datas, 1):
            print(f"\r{category}:{name[:-4]} {num}", end="")
            analyze_body = {"text": data, "analyzer": "sudachi_analyzer"}
            sudachi_morp = es.indices.analyze(index=index_name, body=analyze_body)["tokens"]
            sudachi_token = " ".join([word["token"] for word in sudachi_morp])
            body = {
                "category": category,
                "title": name[:-4],
                "text": data,
                "sudachi": sudachi_token
            }
            es.index(index=index_name, body=body)
    print()

In [11]:
for category in data_dict:
    LINE(f"\n{category} start")
    input_datas(es, es_name, category, data_dict[category])
    print(f"finished {category}")
    LINE(f"\n{category} end")

メディアミックス:ジュラシック・パーク 45バース 5 27 697
finished メディアミックス
アニメ:がきデカ 305fe 8レラガールズ (アニメ) 34ana Famiglia- 138
finished アニメ
公開企業:カーリットホールディングス 6ン 2116ク・カンパニー 6
finished 公開企業
食品:ハーリング (料理) 940゙ 10
finished 食品
アーティスト:ボストン (バンド) 15ニー 6・バンド 12 6ates of America 4
finished アーティスト
おもちゃ:メリーゴーラウンド 86ラー 5
finished おもちゃ
ゲーム:シャドウハンターズ 12ンカードゲーム 1
finished ゲーム
バイク:デグナー 30 25 607ーグ) 23on 9の科学者・成海朔の挑戦〜 6ER 3
finished バイク
ドリンク:ポスカ (飲料) 5 8スキー) 11
finished ドリンク
自動車:ベントレー・4¼リットル 954 24 4
finished 自動車


In [12]:
LINE(f"おわったよ")

In [13]:
for category in data_dict:
    print(category)
    body={
        "query": {
            "term": {
                "category": category
            }
        }
    }
    result = es.search(
            index=es_name,
            body=body
    )
    print('total', result['hits']['total'])

メディアミックス
total {'value': 5812, 'relation': 'eq'}
アニメ
total {'value': 4030, 'relation': 'eq'}
公開企業
total {'value': 2721, 'relation': 'eq'}
食品
total {'value': 657, 'relation': 'eq'}
アーティスト
total {'value': 4953, 'relation': 'eq'}
おもちゃ
total {'value': 516, 'relation': 'eq'}
ゲーム
total {'value': 814, 'relation': 'eq'}
バイク
total {'value': 10000, 'relation': 'gte'}
ドリンク
total {'value': 154, 'relation': 'eq'}
自動車
total {'value': 372, 'relation': 'eq'}


In [18]:
for category in data_dict:
    LINE(f"\n{category} start")
    response = es.search(
                    scroll='2m',
                    size=10000,
                    index=es_name,
                    body={
                        "query": {
                            "term": {
                                "category": category
                            }
                        }
                    }
                )
    sid = response['_scroll_id']
    # print('sid', sid)
    # print('total', response['hits']['total'])

    scroll_size = len(response['hits']['hits'])
    # print('scroll_size', scroll_size)
    open(f"./sudachi_dataset/{category}.txt", mode="x", encoding="utf-8")
    counter = 1
    while True:
        # スクロールサイズ 0 だったら終了
        if scroll_size <= 0:
            break

        # 検索結果を処理
    #     get_doc(response['hits']['hits'])
        hits = response['hits']['hits']
        for hit in hits:
            text = hit['_source']['sudachi']
            text = re.sub(r"([0-9]+) (年) ", r"\1\2 ", text)
            if len(text.split()) < 10 :
                continue
    #             print(f"{counter} {text}", end="")
            with open(f"./sudachi_dataset/{category}.txt", mode="a", encoding="utf-8") as save_file:
                save_file.write(text + "\n")
            counter += 1

        # スクロールから次の検索結果取得
        response = es.scroll(scroll_id=sid, scroll='10m')
        scroll_size = len(response['hits']['hits'])
    #     print('scroll_size', scroll_size)
    LINE(f"\n{category} end")

In [ ]:
counter = 1
while True:
    # スクロールサイズ 0 だったら終了
    if scroll_size <= 0:
        break

    # 検索結果を処理
#     get_doc(response['hits']['hits'])
    hits = response['hits']['hits']
    for hit in hits:
        text = hit['_source']['sudachi']
        print(f"\r{counter} {text}", end="")
        counter += 1

    # スクロールから次の検索結果取得
    response = es.scroll(scroll_id=sid, scroll='10m')
    scroll_size = len(response['hits']['hits'])
#     print('scroll_size', scroll_size)

In [ ]:
result